In [1]:
import pandas as pd
import numpy as np
import time

In [4]:
df = pd.read_csv("adult.csv")

In [10]:
sencitive_attribute = 'income'
df

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [8]:
c_thresold = 0.7
l_thresold = 2
min_group_size = 30

In [21]:
def recervie_partaions(df_group,c_thresold,l_thresold,min_group_size):
    
    sens_counts =  df_group[sencitive_attribute].value_counts(normalize = True)
    max_conf = sens_counts.max() if not sens_counts.empty else 0
    num_disticnt = df_group[sencitive_attribute].nunique()

    if max_conf <= c_thresold and num_disticnt >= l_thresold:
        return [df_group]

    if len(df_group) < min_group_size:
        return []


    df_group = df_group.sort_values("age")
    min_index = len(df_group)//2
    group1 = df_group.iloc[:min_index]
    group2 = df_group.iloc[min_index:]


    group1 = recervie_partaions(group1,c_thresold,l_thresold,min_group_size)
    group2 = recervie_partaions(group2,c_thresold,l_thresold,min_group_size)

    return group1 + group2;

In [22]:
start_time = time.time()
final_group = recervie_partaions(df,c_thresold,l_thresold,min_group_size)
end_time = time.time()

In [24]:
if final_group:
    df_anonymized = pd.concat(final_group)
else:
    df_anonymized = pd.DataFrame(columns=df.columns)

original_size = len(df)
anonymized_size = len(df_anonymized)
supprased_records = original_size - anonymized_size


group_sizes = [len(g) for g in final_group]
num_groups = len(final_group)

In [28]:
print(f"Original Dataset size : {original_size} ")
print(f"Anonymized Dataset size : {anonymized_size}")
print(f"Supprased records :{supprased_records}")
print(f"Number of final equivalnce class: {num_groups}")
if group_sizes:
    print("Equivalence Class Size Distribution:")
    print(pd.Series(group_sizes).describe())

print(f"Total Processing Time: {end_time - start_time:.4f} seconds")

# (Optional) Save the anonymized dataset.
df_anonymized.to_csv("adult_recursive_cl_anonymized.csv", index=False)
print("\nFinal anonymized dataset saved as 'adult_recursive_cl_anonymized.csv'")

Original Dataset size : 32561 
Anonymized Dataset size : 19478
Supprased records :13083
Number of final equivalnce class: 83
Equivalence Class Size Distribution:
count       83.000000
mean       234.674699
std       1786.319777
min         15.000000
25%         16.000000
50%         16.000000
75%         31.000000
max      16281.000000
dtype: float64
Total Processing Time: 0.7679 seconds

Final anonymized dataset saved as 'adult_recursive_cl_anonymized.csv'
